In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BinaryCNN(nn.Module):
    def __init__(self):
        super(BinaryCNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)

        # Calculate the size after convolutions and pooling
        # Input size: 256x256
        # After first pool: 128x128
        # After second pool: 64x64
        # After third pool: 32x32
        self.fc_input_dim = 64 * 32 * 32

        # Fully connected layers
        self.fc1 = nn.Linear(self.fc_input_dim, 128)
        self.fc2 = nn.Linear(128, 1)  # Output layer for binary classification

    def forward(self, x):
        # x shape: (batch_size, 3, 256, 256)
        x = F.relu(self.conv1(x))
        x = self.pool(x)  # 128x128
        x = F.relu(self.conv2(x))
        x = self.pool(x)  # 64x64
        x = F.relu(self.conv3(x))
        x = self.pool(x)  # 32x32

        x = x.view(-1, self.fc_input_dim)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x  # Sigmoid for binary output

In [20]:
from torch.utils.data import Dataset, DataLoader
import cv2

class LabelDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_filenames = os.listdir(image_dir)
        self.transform = transform

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_filenames[idx])

        image = cv2.imread(img_name)

        image = cv2.resize(image, (256, 256))

        # Convert to PyTorch tensors and permute to (C, H, W)
        if self.transform:
            image = self.transform(image)
        else:
            # Convert to tensor and permute channels if no transform
            image = image.astype('float32') / 255.0
            image = torch.from_numpy(image).permute(2, 0, 1)

        # Assign label based on filename
        filename_lower = self.image_filenames[idx].lower()
        if "alive" in filename_lower:
          label = 1
        elif "dead" in filename_lower:
          label = 0
        else:
          label = -1

        return image, label

In [26]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

model = BinaryCNN()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

# Directories
image_dir = '/content/drive/My Drive/Research/Labeled Items'


# Create Dataset and DataLoader
dataset = LabelDataset(image_dir)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

for epoch in range(15):
    model.train()
    running_loss = 0.0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.float().to(device)  # Convert labels to float for BCELoss

        optimizer.zero_grad()
        outputs = model(images).view(-1) # Remove extra dimensions
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    print(f"Epoch [{epoch+1}/15] Training Loss: {epoch_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'binary_cnn.pth')

Epoch [1/15] Training Loss: 1.5934
Epoch [2/15] Training Loss: 0.7403
Epoch [3/15] Training Loss: 0.6795
Epoch [4/15] Training Loss: 0.6719
Epoch [5/15] Training Loss: 0.6435
Epoch [6/15] Training Loss: 0.6288
Epoch [7/15] Training Loss: 0.5534
Epoch [8/15] Training Loss: 0.4661
Epoch [9/15] Training Loss: 0.3631
Epoch [10/15] Training Loss: 0.3215
Epoch [11/15] Training Loss: 0.1697
Epoch [12/15] Training Loss: 0.1477
Epoch [13/15] Training Loss: 0.1143
Epoch [14/15] Training Loss: 0.0925
Epoch [15/15] Training Loss: 0.1013


In [30]:
# Validation

image_dir = '/content/drive/My Drive/Research/Train Label'

trainset = LabelDataset(image_dir)
dataloader = DataLoader(trainset, batch_size=10, shuffle=True)

model.eval()
val_loss = 0.0
correct = 0
with torch.no_grad():
    for images, labels in dataloader:
          images, labels = images.to(device), labels.float().to(device)
          outputs = model(images).view(-1)
          loss = criterion(outputs, labels)
          val_loss += loss.item() * images.size(0)

          # Calculate accuracy
          preds = (outputs >= 0.5).float()
          correct += (preds == labels).sum().item()

    val_loss /= len(dataloader.dataset)
    accuracy = correct / len(dataloader.dataset) * 100
    print(f"Validation Loss: {val_loss:.4f} | Validation Accuracy: {accuracy:.2f}%\n")

Validation Loss: 0.3550 | Validation Accuracy: 91.67%

